In [4]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 544.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 62.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 514.1 kB/s eta 0:00:0000:0100:01
     ━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.3 MB/s eta 0:00:0000:01m00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4d15e5d773d27d4c598082b7e30ea159e5eb09934328bd098d1dddf71e87374e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
  Created wheel for lit: filename=lit-16.0.6-py3-none-any.whl size=93582 sha256=5dacd9179f282b1fecab7c78cd0c38061668e11f4a19e76a7ff337940d231323
  Stored in directory: /home/ec2-

In [37]:
import pandas as pd
import boto3
import io
from tqdm import tqdm
import numpy as np
from sagemaker import KMeans
import sagemaker
from sagemaker import get_execution_role

In [2]:
# S3에 액세스할 수 있는 클라이언트 생성
s3 = boto3.client('s3')

# S3에서 파일 가져오기
response = s3.get_object(Bucket='dothis-ai-test', Key='data/channel_keyword_data_20230616.csv')

# pandas로 read
df = pd.read_csv(io.BytesIO(response["Body"].read()))

df

,channel_name,tf-idf-keywords
0,G마켓 옥션 판매자 교육센터 TV,"마켓,옥션,창업,오픈,광고,tv,판매,상품,ft,esm"
1,여의도 세력개미,"전망,주가,세력,긴급,목표,속보,매집,분석,공개,리튬"
2,정수TV,"학년,초등,학교,차시,설명,학기,영어,대교,수학,사회"
3,주식25,"주가,전망,주주,대박,매수,호재,상한가,축하,당장,필수"
4,나인트롯,"트롯,tv,조선,미스터,시청,홍지윤,방송,화요일,확정,프로그램"
...,...,...
6384,무쏘TV-김희천 대표,"가치주,금리,트윗,테마,중국,주식,종목,인상,육계,원전"
6385,심심한 회사원,"리얼돌,엄마"
6386,미정살롱,"교육,드라이,볼륨,사모님,인턴"
6387,내맘대로2,"shorts,결혼,권해,스킨,아내,윤지민"


In [23]:
data = list(set(",".join(df["tf-idf-keywords"].tolist()).split(",")))
len(data)

train_data = data[:-100]
test_data = data[-100:]
len(data), len(train_data), len(test_data)

(14141, 14041, 100)

In [38]:
# SageMaker 실행 역할 가져오기
role = get_execution_role()

# SageMaker 세션 생성
sagemaker_session = sagemaker.Session()


In [40]:
# K-Means estimator 생성
bucket = 'dothis-ai-test'
kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.m5.xlarge',
                output_path='s3://{}/models'.format(bucket),
                k=71,
                feature_dim=n_components,
                base_job_name='kmeans-keyword-test-1')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [41]:
# K-Means 모델 훈련

# 학습 작업 실행
kmeans.fit(kmeans.record_set(train_lsa_word_vectors))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-keyword-test-1-2023-06-16-07-55-55-451


2023-06-16 07:55:55 Starting - Starting the training job...
2023-06-16 07:56:10 Starting - Preparing the instances for training......
2023-06-16 07:57:08 Downloading - Downloading input data............
2023-06-16 07:59:25 Training - Training image download completed. Training in progress.
2023-06-16 07:59:25 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[06/16/2023 07:59:19 INFO 140387669366592] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_profiler': 'false', '_kvstore': 'a


2023-06-16 07:59:35 Completed - Training job completed
Training seconds: 147
Billable seconds: 147


In [42]:
# 추론 엔드포인트 생성
endpoint_name = 'kmeans-keyword-inference-test-1'  # SageMaker 엔드포인트 이름

kmeans.deploy(initial_instance_count=1, 
              instance_type='ml.m5.xlarge',
              endpoint_name=endpoint_name)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: kmeans-keyword-test-1-2023-06-16-08-01-31-535
INFO:sagemaker:Creating endpoint-config with name kmeans-keyword-inference-test-1
INFO:sagemaker:Creating endpoint with name kmeans-keyword-inference-test-1


----!

In [44]:
# 추론 수행

vectors = [sentence_model.encode(vector) for vector in tqdm(test_data)]
# vectors = [sentence_model.encode(vector) for vector in test_data]

word_vectors = np.vstack(vectors)

  0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:00<00:07, 13.07it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 98%|█████████▊| 98/100 [00:07<00:00, 13.01it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:08<00:00, 12.37it/s]


In [45]:
n_components = 100
svd = TruncatedSVD(n_components=n_components)
test_lsa_word_vectors = svd.fit_transform(word_vectors)
test_lsa_word_vectors.shape

(100, 100)

In [67]:
# 입력 데이터를 CSV 형식으로 변환
input_data = pd.DataFrame(test_lsa_word_vectors).astype('float32').to_csv(header=False, index=False)

In [73]:
# 추론 요청
runtime_client = boto3.client('sagemaker-runtime')
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=input_data)

In [74]:
# 추론 결과 파싱
import ast


result = response['Body'].read().decode('utf-8')

# 문자열을 딕셔너리로 변환
dictionary = ast.literal_eval(result)
result = dictionary["predictions"]

In [77]:
result

[{'closest_cluster': 65.0, 'distance_to_cluster': 8.459484100341797},
 {'closest_cluster': 26.0, 'distance_to_cluster': 9.320382118225098},
 {'closest_cluster': 5.0, 'distance_to_cluster': 8.602652549743652},
 {'closest_cluster': 36.0, 'distance_to_cluster': 8.018691062927246},
 {'closest_cluster': 50.0, 'distance_to_cluster': 9.17749309539795},
 {'closest_cluster': 35.0, 'distance_to_cluster': 8.719600677490234},
 {'closest_cluster': 50.0, 'distance_to_cluster': 9.772226333618164},
 {'closest_cluster': 60.0, 'distance_to_cluster': 9.646796226501465},
 {'closest_cluster': 4.0, 'distance_to_cluster': 8.81623363494873},
 {'closest_cluster': 19.0, 'distance_to_cluster': 10.057835578918457},
 {'closest_cluster': 31.0, 'distance_to_cluster': 8.950265884399414},
 {'closest_cluster': 48.0, 'distance_to_cluster': 9.799546241760254},
 {'closest_cluster': 22.0, 'distance_to_cluster': 9.913366317749023},
 {'closest_cluster': 8.0, 'distance_to_cluster': 9.24266242980957},
 {'closest_cluster': 3.0,

In [86]:
test_result = pd.DataFrame()
test_result['word'] = test_data
test_result['result'] = [i['closest_cluster'] for i in result]
test_result

,word,result
0,부실,65.0
1,동구,26.0
2,콘깬,5.0
3,패턴,36.0
4,와룡,50.0
...,...,...
95,할리,23.0
96,파이버,5.0
97,전립선,44.0
98,최성재,54.0


In [52]:
len(train_data), len(train_lsa_word_vectors)

(14041, 14041)

In [92]:
test_result[test_result["result"] == 3]['word']

14     마술
56    역발상
92     예문
Name: word, dtype: object

In [93]:
# SageMaker 클라이언트 생성
sagemaker_client = boto3.client('sagemaker')

# 엔드포인트 삭제 요청
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '23408fe8-89dd-4133-98e4-19522aa39283',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '23408fe8-89dd-4133-98e4-19522aa39283',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 16 Jun 2023 09:05:14 GMT'},
  'RetryAttempts': 0}}